In [16]:
import os
import sys
import importlib
sys.path.append("../")
get_features_and_label = importlib.import_module('01-feature-engineering').get_features_and_label



In [18]:
import json

# All UserTransactions Dataframe

First we get all the userTransactions from the api. 

These events are saved in a file called `all_events.json`

We can create a dataframe of all userTransactions like so:



In [24]:
# check for the data file
print("checking for event data on disk ...")
data_file = os.getcwd() + '/../data/all_events.json' # get file location
data_file # str


checking for event data on disk ...


'/home/j3/code/defi_credit_score/notebooks/../data/all_events.json'

In [26]:
# load the all events.json from disk to an object we can use

if os.path.isfile(data_file):
    print("\"all_events.json\" found, loading from disk.")
    with open(data_file) as f:
        all_events = json.load(f)


"all_events.json" found, loading from disk.


In [28]:
# now we have access to all_events
len(all_events)


461549

In [29]:
# a single event
all_events[0]

{'timestamp': 1627952617,
 'user_id': '0x1eebec3312d9345c09b1a74fb7502a68660c089b',
 'txn_id': '0x452aee6d672f5be5aa3221def15850d2a9598ec137e439ed01747276d067b46d:3',
 'event_type': 'unknown'}

In [30]:
# getting all events into a dataframe
import pandas as pd
user_transactions = pd.DataFrame(all_events)


In [31]:
user_transactions

,timestamp,user_id,txn_id,event_type,amount,pool_id,pool_lendingPool,reserve_id,reserve_symbol,accruedBorrowInterest,borrowRate,borrowRateMode,amountAfterFee,fee,collateralAmount,collateralReserve_id,collateralReserve_underlyingAsset,liquidator,principalAmount,principalReserve_id,principalReserve_underlyingAsset
0,1627952617,0x1eebec3312d9345c09b1a74fb7502a68660c089b,0x452aee6d672f5be5aa3221def15850d2a9598ec137e4...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1627947707,0x5b2a89ee094ec6778ed72be05f0e219b956dbbb0,0x5b0d2ed0320217225b62811617a09b65c799db05bf67...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1627947497,0x1eebec3312d9345c09b1a74fb7502a68660c089b,0x60b56991a2d6c23ff41e13431904bb43f3182b31ac2b...,deposit,1218284692576855568525,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x6b175474e89094c44da98b954eedeac495271d0f0x24...,DAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1627945331,0x1eebec3312d9345c09b1a74fb7502a68660c089b,0x6b805a7d536af417fc23aca2f3f8f59b84ac408aadb3...,deposit,224709657539875960368546,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x6b175474e89094c44da98b954eedeac495271d0f0x24...,DAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1627936585,0x39132c7f68616ffd34dba3cacfd1635f3b64ddf6,0x77ed6a261db0a0a67b24b81953a3a8f37aa140aeff43...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461544,1578505466,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0x5cf2cad66b0fbb6e2d32f31addb11443a5c54887f786...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461545,1578505416,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0x0ae2a6f2883a25bc5fecd429460a37938a5edfb4aad7...,deposit,63053681658949241179,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xe41d2489571d322189246dafa5ebde1f4699f4980x24...,ZRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461546,1578505074,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0x9b02ca82d622b4cc82e77f42d66901df9dc4d8f4d973...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461547,1578504936,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0x75019cc109721c59bf70a4a51f40f94fedfe06dbb9aa...,deposit,42731,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x2260fac5e5542a773aa44fbcfedf7c193bc2c5990x24...,WBTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
user_transactions.shape

(461549, 21)

In [33]:
# all borrow events
user_transactions[user_transactions["event_type"] == "borrow"]

,timestamp,user_id,txn_id,event_type,amount,pool_id,pool_lendingPool,reserve_id,reserve_symbol,accruedBorrowInterest,borrowRate,borrowRateMode,amountAfterFee,fee,collateralAmount,collateralReserve_id,collateralReserve_underlyingAsset,liquidator,principalAmount,principalReserve_id,principalReserve_underlyingAsset
7,1627934303,0x21ce16237406a6e731e32503639ddd5992f23d6a,0xf0a71e23d4505879bda91c7f088e912f3846bdee6885...,borrow,10000000000,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480x24...,USDC,3375240,45578886540815726061619900,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1627896917,0x473869d211a9364b13806b97882f741acc444295,0x9276c4240f481cbe03cb5072593744c8e41353cce8dc...,borrow,3000000000000000000,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee0x24...,ETH,3751967947903407,2976595229856732580413260,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,1627861398,0x440efbba3f3d971fd8411c580197ecb53ef71f87,0xca62d0cac156472b930b440575ca85e9f4cf84a16636...,borrow,15000000000000000000000,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x0000000000085d4780b73119b644ae5ecd22b3760x24...,TUSD,35900666116638245794,28538409343919156047134532,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,1627856070,0x738726961dabef11514ec9b75e0665ee0050ebc4,0xabada446ef02a776217064061d4bbe199b8270ddbd40...,borrow,33983728169,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480x24...,USDC,54042754,44037780200781626547534390,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,1627833352,0x0d9f4d0dabbfa76a5ccec6a1542326ca3da57ae5,0x22afeb0053ffcd56670b94257d43f7cd186fd08799f1...,borrow,2000000000000000000000,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x0000000000085d4780b73119b644ae5ecd22b3760x24...,TUSD,569941422291171815,28511517660919015923889376,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461527,1578506443,0x5d3183cb8967e3c9b605dc35081e5778ee462328,0x9b327bdcb9576a134446297268150e5aac4ea165242b...,borrow,100000000000000000000,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x0000000000085d4780b73119b644ae5ecd22b3760x24...,TUSD,0,10250000000000000000000000,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461529,1578506331,0x5d3183cb8967e3c9b605dc35081e5778ee462328,0x78d1eb85882c86e19e4447e028189fd512684440cfa2...,borrow,100000000,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480x24...,USDC,0,11666666666666666666666667,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461532,1578506080,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0x095e6a129dff7757b6ad3b706edcc3c2fe9c5435b83c...,borrow,12347142096305911092,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xe41d2489571d322189246dafa5ebde1f4699f4980x24...,ZRX,0,9790976332746364637143552,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461533,1578506041,0x5d3183cb8967e3c9b605dc35081e5778ee462328,0xf02364679b70045cacd1f21d8345bef39afa1390bb97...,borrow,1000000000000000000000,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x6b175474e89094c44da98b954eedeac495271d0f0x24...,DAI,0,33861572913303013691138106,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# all repay events: 
user_transactions[user_transactions["event_type"] == "repay"]

,timestamp,user_id,txn_id,event_type,amount,pool_id,pool_lendingPool,reserve_id,reserve_symbol,accruedBorrowInterest,borrowRate,borrowRateMode,amountAfterFee,fee,collateralAmount,collateralReserve_id,collateralReserve_underlyingAsset,liquidator,principalAmount,principalReserve_id,principalReserve_underlyingAsset
21,1627879609,0xd4bce9c082e315b8e3d0a79bfb5c6daa36e9531b,0x98a12679af49ec57f7660ea2d44e4cf9f65d486f109b...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xdac17f958d2ee523a2206206994597c13d831ec70x24...,USDT,NaN,NaN,NaN,21923902804,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,1627866040,0xfb5c17f3941bbde978a7c47ad03be737f2003489,0x91a85ba8018a6284529ad1c9955e5c90311d789eeab9...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee0x24...,ETH,NaN,NaN,NaN,999999844420591181,155579408819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,1627860470,0x738726961dabef11514ec9b75e0665ee0050ebc4,0x607e8f3d8f3e4ae82321d41c414a9a58a32aa7991100...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480x24...,USDC,NaN,NaN,NaN,33650539866,39909,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,1627859752,0x745d939cad45d371a9b136d32b959cb5c7005f84,0xa19b717c79ee31ab403562b80553e5b6913943678187...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x0000000000085d4780b73119b644ae5ecd22b3760x24...,TUSD,NaN,NaN,NaN,2469026948490738010424,63447483838684129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,1627856243,0x745d939cad45d371a9b136d32b959cb5c7005f84,0xc14d59ccab0915017e8e3c8468d8d824a5e3aa766716...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480x24...,USDC,NaN,NaN,NaN,4330508687,412,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461482,1578517035,0xf4d9a2d0075e0f003410ae774456fbeeedd18c72,0x513502a6a21c5dc6c119ecda6e2241b85f0ee345f373...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x514910771af9ca656af840dff83e8264ecf986ca0x24...,LINK,NaN,NaN,NaN,140471701523504553,352059402314548,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461522,1578507381,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0xd86fdea9b34c33ebf3a14c6a4763476a998e469effe3...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a20x24...,MKR,NaN,NaN,NaN,3064977088240704,15363292539158,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461530,1578506250,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0x84b8da004013a8bcad45404420a840bccef94c5c5f5e...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f0x24...,SNX,NaN,NaN,NaN,1071708062621108150,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461531,1578506093,0x6b4fc9c327eca193f7d21b0202e68c5df5eafaa6,0x05bb2b6ae213c3c0ff5207591f347b0473d74c4265d2...,repay,NaN,0x24a42fd28c976a61df5d00d0599c34c4f90748c8,0x398ec7346dcd622edc5ae82352f02be94c62d119,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f0x24...,SNX,NaN,NaN,NaN,1428944850306553904,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Next: The model use case interface: 

tl:dr: user_address -> api, 
api -> model, model -> score -> user

Let's imagine how our user would interface with our service. Let's say they upload an address to a form, and when they do so, our api would then pull all their past transactions from Aave, run our risk model, and given the output from the model we'll return a credit score.

(For the sake of this excercise, let's say we are on a v.1 of the product, so we'll just return a probability they will default. Higher probability is analogous to a lower credit score.) 



# Next: Get a profile for a specific user

For this excercise, let's plan to give a credit score to a user based on their previous history. 

To do this, we'll want to get all of a user's past transactions:

# We'll use "get all_user_mapping.json" that we returned from the fetcher. 

```python
This gives us a nice mapping for a user and all their given events
"""
{
    user1_id: [
        {event1}
        {event2}
        {event3}
        {event4}
    ]
    user2_id:        
        {event1}
        {event2}
        {event3}
        {event4}
}
"""
```


In [37]:
### Let's load this mapping from disk

# all user mapping.json
# check for the data file
print("checking for event data on disk ...")
data_file = os.getcwd() + '/../data/all_user_mapping.json' # get file location
# data_file # str

if os.path.isfile(data_file):
    print("\"all_user_mapping.json\" found, loading from disk.")
    with open(data_file) as f:
        user_event_mapping = json.load(f)

checking for event data on disk ...
"all_user_mapping.json" found, loading from disk.


In [39]:
# we have a dictionary
type(user_event_mapping)

dict

In [44]:
# the keys are user ids, here are the first 20
list(user_event_mapping)[:20]


['0x1eebec3312d9345c09b1a74fb7502a68660c089b',
 '0x5b2a89ee094ec6778ed72be05f0e219b956dbbb0',
 '0x39132c7f68616ffd34dba3cacfd1635f3b64ddf6',
 '0x21ce16237406a6e731e32503639ddd5992f23d6a',
 '0x959237c226ba652196be8641a7d497b6fd4bbb33',
 '0xdac09f37e132d91b962f30e6ec40d2d08b82b0fa',
 '0xbf609e91ffedc7e5cf9704d9f3c0acea81126d21',
 '0x6f38525a99a2a6dcd40be420cf752b13178058df',
 '0x95ed6f043bbc9fbf604a6879869d03b31686294d',
 '0x473869d211a9364b13806b97882f741acc444295',
 '0xdda2d57aa2cb55af90e42cc8c71f6c63fae58a56',
 '0x34c8c9d9d4c5abe245df85c159a566322840b6a5',
 '0x66f54006b47d1b381b5b933f2de9bf05a6b4eb40',
 '0xd255f83c6fedb746763fbc86f4476c46d114feee',
 '0xc176761d388caf2f56cf03329d82e1e7c48ae09c',
 '0xd4bce9c082e315b8e3d0a79bfb5c6daa36e9531b',
 '0x6067b8470c1cea51c4fdac7afc53f50c8b122ebe',
 '0xfb5c17f3941bbde978a7c47ad03be737f2003489',
 '0x05d534f28700ba8f749a403e181c557fda6888fe',
 '0x745d939cad45d371a9b136d32b959cb5c7005f84']

In [59]:
# let's see the event list for a given user

print(f"number of events for user id 0x1eebec3312d9345c09b1a74fb7502a68660c089b:\n{len(user_event_mapping['0x1eebec3312d9345c09b1a74fb7502a68660c089b'])}")


number of events for user id 0x1eebec3312d9345c09b1a74fb7502a68660c089b:
105


In [61]:
# let's take a look at the first 5 events for this user
user_event_mapping['0x1eebec3312d9345c09b1a74fb7502a68660c089b'][:5]



[{'timestamp': 1627952617,
  'user_id': '0x1eebec3312d9345c09b1a74fb7502a68660c089b',
  'txn_id': '0x452aee6d672f5be5aa3221def15850d2a9598ec137e439ed01747276d067b46d:3',
  'event_type': 'unknown'},
 {'amount': '1218284692576855568525',
  'pool_id': '0x24a42fd28c976a61df5d00d0599c34c4f90748c8',
  'pool_lendingPool': '0x398ec7346dcd622edc5ae82352f02be94c62d119',
  'reserve_id': '0x6b175474e89094c44da98b954eedeac495271d0f0x24a42fd28c976a61df5d00d0599c34c4f90748c8',
  'reserve_symbol': 'DAI',
  'timestamp': 1627947497,
  'user_id': '0x1eebec3312d9345c09b1a74fb7502a68660c089b',
  'txn_id': '0x60b56991a2d6c23ff41e13431904bb43f3182b31ac2b9a2809748e1b35d84725:1',
  'event_type': 'deposit'},
 {'amount': '224709657539875960368546',
  'pool_id': '0x24a42fd28c976a61df5d00d0599c34c4f90748c8',
  'pool_lendingPool': '0x398ec7346dcd622edc5ae82352f02be94c62d119',
  'reserve_id': '0x6b175474e89094c44da98b954eedeac495271d0f0x24a42fd28c976a61df5d00d0599c34c4f90748c8',
  'reserve_symbol': 'DAI',
  'timesta

# Key next step: getting past events for a user

We see for every event, we have a timestamp. We can imagine this event would be the point a user is applying for a credit score. 


With this "present time" event, we can use the timestamp to get all past events, (and future events) for a user




In [ ]:
# from the 01-feature-engineering.py script



